\Benjamin Bierlein 7641866 Econ 125 Spring 2025 HW 5

a)

In [ ]:
## Import necessary packages
import numpy as np  # Useful for math calculations
import pandas as pd  # Useful for data manipulation
import statsmodels.api as sm  # Useful for regression analysis
import statsmodels.formula.api as smf  # Required for regression using formulas
import matplotlib.pyplot as plt  # Useful for plotting

# Load Data
vac_df = pd.read_csv("/content/vacation.csv")
print(vac_df.head())

# Fit OLS model with robust standard errors
ols_model = smf.ols('miles ~ income + age + kids', data=vac_df).fit(cov_type='HC1')

# Extract OLS estimates and robust standard errors
ols_summary = ols_model.summary()
print(ols_summary)

# Print robust SEs next to coefficients
print("\n=== OLS Coefficients with Robust Standard Errors (HC1) ===")
for param, coef, robust_se in zip(
    ols_model.params.index,
    ols_model.params.values,
    ols_model.bse.values
):
    print(f"{param:>10}: Coef = {coef:>8.3f}, Robust SE = {robust_se:>8.3f}")


   miles  income  age  kids
0    902      41   26     0
1    491      31   38     3
2   1841      87   40     2
3    406      54   48     4
4      0      77   43     4
                            OLS Regression Results                            
Dep. Variable:                  miles   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.331
Method:                 Least Squares   F-statistic:                     44.66
Date:                Sat, 17 May 2025   Prob (F-statistic):           4.86e-22
Time:                        03:33:03   Log-Likelihood:                -1504.6
No. Observations:                 200   AIC:                             3017.
Df Residuals:                     196   BIC:                             3030.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          

b)

In [ ]:
# Define missing squared and interaction terms explicitly
vac_df['income_sq'] = vac_df['income'] ** 2
vac_df['age_sq'] = vac_df['age'] ** 2
vac_df['kids_sq'] = vac_df['kids'] ** 2

vac_df['income_age'] = vac_df['income'] * vac_df['age']
vac_df['income_kids'] = vac_df['income'] * vac_df['kids']
vac_df['age_kids'] = vac_df['age'] * vac_df['kids']

# Re-run variance regression
variance_formula = 'residuals_sq_log ~ income + age + kids + income_sq + age_sq + kids_sq + income_age + income_kids + age_kids'
var_model = smf.ols(variance_formula, data=vac_df).fit()

# Get predicted variances and calculate weights
predicted_log_var = var_model.fittedvalues
weights = np.exp(-predicted_log_var)

# FGLS model using WLS
fgls_model = smf.wls('miles ~ income + age + kids', data=vac_df, weights=weights).fit()

# Format results
fgls_results = pd.DataFrame({
    "Coefficient": fgls_model.params,
    "Std. Error": fgls_model.bse
})

print(fgls_model.summary())


                            WLS Regression Results                            
Dep. Variable:                  miles   R-squared:                       0.659
Model:                            WLS   Adj. R-squared:                  0.654
Method:                 Least Squares   F-statistic:                     126.1
Date:                Tue, 13 May 2025   Prob (F-statistic):           1.58e-45
Time:                        23:15:49   Log-Likelihood:                -1483.9
No. Observations:                 200   AIC:                             2976.
Df Residuals:                     196   BIC:                             2989.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -389.2648     76.911     -5.061      0.0